----

In [1]:
#    GNU LESSER GENERAL PUBLIC LICENSE
#    Version 3, 29 June 2007
#    Copyright (C) 2007 Free Software Foundation, Inc. <http://fsf.org/>
#    Everyone is permitted to copy and distribute verbatim copies
#    of this license document, but changing it is not allowed.

#    James Gaboardi, 2016

-----

## Spatial Optimization Model Building in Python: Cplex v. Gurobi
## *p*-Median Problem Holding *p* Constant 
## Increasing Matrix Dimensions from 2x2 to 200x200

----

#### James D. Gaboardi &nbsp;&nbsp; |  &nbsp;&nbsp; Florida State University &nbsp; &nbsp;|  &nbsp;&nbsp; Department of Geography 

----


In [2]:
# Local path on user's machine
path = '/Users/jgaboardi/Desktop/CPLEX_v_Gurobi/Data/'

# Imports
import pysal as ps
import geopandas as gpd
import numpy as np
import networkx as nx
import shapefile as shp
from shapely.geometry import Point
import shapely
from collections import OrderedDict
import pandas as pd
import qgrid
import cplex as cp
import gurobipy as gbp
import time
import bokeh
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.io import output_notebook, output_file, show
from bokeh.models import (HoverTool, BoxAnnotation, GeoJSONDataSource, 
                          GMapPlot, GMapOptions, ColumnDataSource, Circle, 
                          DataRange1d, PanTool, WheelZoomTool, BoxSelectTool,
                          ResetTool, MultiLine)
import utm
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
output_notebook()

/Users/jgaboardi/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Loading BokehJS ...

In [3]:
N = 200  # Matrix Dimensions

# PANDAS DATAFRAME OF p/y results
n_list = []
for i in range(2, N+1):
    matrix = 'n='+str(i)+'x'+str(i)
    n_list.append(matrix)
y_list = []
for i in range(1, N+1):
    y = 'y'+str(i)
    y_list.append(y)

In [4]:
solve_df = pd.DataFrame(index=n_list)

In [5]:
# PMP
VAL_PMP_cplex = []
AVG_PMP_cplex = []
VAL_PMP_gurobi = []
AVG_PMP_gurobi = []
solve_time_cplex = []
solve_time_gurobi = []

In [6]:
for n in range(2, N+1): #--------- n Matrix Dimensions
    
    np.random.seed(850)
    
    # Client Weights for demand
    Ai = np.random.randint(1, 5, n)
    Ai = Ai.reshape(len(Ai),1)
    AiSum = np.sum(Ai) # Sum of Weights (Total Demand)
    
    # Cost Matrix
    Cij = np.random.randint(1, 5, n*n)
    Cij = Cij.reshape(n,n)
    Sij = Ai * Cij
    client_nodes = range(len(Sij))
    service_nodes = range(len(Sij[0]))
    all_nodes_len = len(Sij) * len(Sij[0])
    ALL_nodes_range = range(all_nodes_len)

    ##############################################################
    # CPLEX
    t1_PMP_cplex = time.time()

    np.random.seed(352)

    m = cp.Cplex()                                      # Create model
    m.parameters.emphasis.mip.set(2)                    # Set MIP emphasis to '2' --> Optimal
    m.set_problem_type(m.problem_type.LP)               # Set problem type
    m.objective.set_sense(m.objective.sense.minimize)   # Objective Function Sense  ==>  Minimize

    client_var = []
    for orig in client_nodes:
            client_var.append([])
            for dest in service_nodes:
                client_var[orig].append('x'+str(orig+1)+'_'+str(dest+1))

    fac_var = []
    for dest in service_nodes:
            fac_var.append([])
            fac_var[dest].append('y' + str(dest+1))


    # Add Variables
    # Add Client Decision Variables
    m.variables.add(names = [client_var[i][j] for i in client_nodes for j in service_nodes],
                            obj = [Sij[i][j] for i in client_nodes for j in service_nodes], 
                            lb = [0] * all_nodes_len, 
                            ub = [1] * all_nodes_len, 
                            types = ['B'] * all_nodes_len)
    # Add Service Decision Variable
    m.variables.add(names = [fac_var[j][0] for j in service_nodes],
                            lb = [0] * len(Sij[0]), 
                            ub = [1] * len(Sij[0]), 
                            types = ['B'] * len(Sij[0]))

    #    3. Add Constraints
    # Add Assignment Constraints
    for orig in client_nodes:       
        assignment_constraints = cp.SparsePair(ind = [client_var[orig][dest] 
                                                for dest in service_nodes],                           
                                                val = [1] * len(Sij[0]))
        m.linear_constraints.add(lin_expr = [assignment_constraints],                 
                                    senses = ['E'], 
                                    rhs = [1]);
    # Add Facility Constraint
    facility_constraint = cp.SparsePair(ind = [fac_var[j][0] for j in service_nodes], 
                                        val = [1.0] * len(Sij[0]))
    m.linear_constraints.add(lin_expr = [facility_constraint],
                                    senses = ['E'],
                                    rhs = [1])
    # Add Opening Constraint
    OC = [[client_var[i][j]] + [fac_var[j][0]] for i in client_nodes for j in service_nodes]
    for i in OC:
        opening_constraints = cp.SparsePair(ind = i, val = [-1.0, 1.0])
        m.linear_constraints.add(lin_expr = [opening_constraints], 
                                    senses = ['G'], 
                                    rhs = [0])

    #    4. Optimize and Print Results
    m.solve()
    t2_PMP_cplex = time.time()-t1_PMP_cplex
    solve_time_cplex.append(t2_PMP_cplex)
    
    m.write(path+'LP_files/CPLEX_Pmedian'+str(n)+'.lp')
    solution = m.solution
    selected_M_cplex = OrderedDict()
    NEW_Records_PMP_cplex = []
    for f in fac_var:
        if 'x' in f[0]:
            pass
        elif solution.get_values(f[0]) > 0 :
            var = '%s' % f[0]
            selected_M_cplex[var]=(u"\u2588")

    # Display solution.
    print '*******************************************************************'
    for f in fac_var:
        if solution.get_values(f[0]) > 0 :
            print 'Facility %s is open' % f[0]

    val_m_cplex = solution.get_objective_value()
    VAL_PMP_cplex.append(round(val_m_cplex, 3))
    avg_m_cplex = float(val_m_cplex)/float(AiSum)
    AVG_PMP_cplex.append(round(avg_m_cplex, 3))
    
    print '*******************************************************************'
    print 'Solution status    = ' , solution.get_status(), ':', solution.status[solution.get_status()]
    print 'Facilities [p]     = ' , len(selected_M_cplex)
    print 'Total cost         = ' , val_m_cplex
    print 'Determination Time = ' , m.get_dettime(), 'ticks'
    print 'Real Time          = ' , t2_PMP_cplex, 'sec.'        
    print 'Matrix Shape       = ' , Sij.shape
    print '*******************************************************************'
    print '\n -- The p-Median Problem -- CPLEX'
    print ' [n] = ', str(n), '\n\n'
    
    
###############################################################################    
    
    #  Gurobi
    
    t1_PMP_gurobi = time.time()
    
    # Instantiate Model
    mPMP = gbp.Model(' -- p-Median -- ')
    # Turn off Gurobi's output
    mPMP.setParam('OutputFlag',False)

    # Add Client Decision Variables (iXj)
    client_var_PMP = []
    for orig in client_nodes:
        client_var_PMP.append([])
        for dest in service_nodes:
            client_var_PMP[orig].append(mPMP.addVar(vtype=gbp.GRB.BINARY,
                                            lb=0,
                                            ub=1,
                                            obj=Sij[orig][dest], 
                                            name='x'+str(orig+1)+'_'+str(dest+1)))

    # Add Service Decision Variables (j)
    serv_var_PMP = []
    for dest in service_nodes:
        serv_var_PMP.append([])
        serv_var_PMP[dest].append(mPMP.addVar(vtype=gbp.GRB.BINARY,
                                      lb=0,
                                      ub=1,
                                      name='y'+str(dest+1)))

    # Update the model
    mPMP.update()

    #     3. Set Objective Function
    mPMP.setObjective(gbp.quicksum(Sij[orig][dest]*client_var_PMP[orig][dest] 
                        for orig in client_nodes for dest in service_nodes), 
                        gbp.GRB.MINIMIZE)


    #     4. Add Constraints
    # Assignment Constraints
    for orig in client_nodes:
        mPMP.addConstr(gbp.quicksum(client_var_PMP[orig][dest] 
                        for dest in service_nodes) == 1)
    # Opening Constraints
    for orig in service_nodes:
        for dest in client_nodes:
            mPMP.addConstr((serv_var_PMP[orig][0] - client_var_PMP[dest][orig] >= 0))

    # Facility Constraint
    mPMP.addConstr(gbp.quicksum(serv_var_PMP[dest][0] for dest in service_nodes) == 1)

    #     5. Optimize and Print Results
    # Solve
    mPMP.optimize()

    # Write LP
    mPMP.write(path+'LP_files/Gurobi_Pmedian'+str(n)+'.lp')
    t2_PMP_gurobi = time.time()-t1_PMP_gurobi
    solve_time_gurobi.append(t2_PMP_gurobi)
    # Record and Display Results
    print '\n*************************************************************************'
    for v in mPMP.getVars():
        if 'x' in v.VarName:
            pass
        elif v.x > 0:
            print 'Facility %s is open' % v.x
    print '\n*************************************************************************'
    selected_M_gurobi = OrderedDict()
    NEW_Records_PMP_gurobi = []
    for v in mPMP.getVars():
        if 'x' in v.VarName:
            pass
        elif v.x > 0:
            var = '%s' % v.VarName
            selected_M_gurobi[var]=(u"\u2588")

    print '    | Selected Facility Locations --------------  ^^^^ '
    print '    | Candidate Facilities [p] ----------------- ', len(selected_M_gurobi)
    val_m_gurobi = mPMP.objVal
    VAL_PMP_gurobi.append(round(val_m_gurobi, 3))
    print '    | Objective Value (miles) ------------------ ', val_m_gurobi
    avg_m_gurobi = float(val_m_gurobi)/float(AiSum)
    AVG_PMP_gurobi.append(round(avg_m_gurobi, 3))
    print '    | Avg. Value / Client (miles) -------------- ', avg_m_gurobi
    print '    | Real Time to Optimize (sec.) ------------- ', t2_PMP_gurobi
    print '    | Matrix Shape ----------------------------- ', Sij.shape
    print '*************************************************************************'
    print '\n -- The p-Median Problem -- Gurobi'
    print ' [n] = ', str(n), '\n\n'

Found incumbent of value 2.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 3 times.
MIP Presolve eliminated 2 rows and 1 columns.
Aggregator did 5 substitutions.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.01 ticks)

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.02 ticks)
Parallel b&c, 4 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.02 ticks)
*******************************************************************
Facility y1 is open
*******************************************************************
Solution status    =  101 : MIP_optimal
Facilities [p]     =  1
Total cost         =  2.0
Determination Time =  0.0195760726929 ticks
Real Time          =  0.0149681568146 sec.
Matrix Shape       =  (2, 2)
*****************************************************************

In [7]:
solve_df.index.name = 'Matrix Size'
solve_df['CPLEX Solution Time (Real Time (sec.))'] = solve_time_cplex
solve_df['Gurobi Solution Time (Real Time (sec.))'] = solve_time_gurobi
qgrid.show_grid(solve_df)

In [12]:
TOOLS = 'wheel_zoom, pan, reset, crosshair, save, hover'

source_cplex = ColumnDataSource(
        data=dict(
            x=range(1, N+1),
            y=solve_time_cplex,
            obj=solve_time_cplex,
            desc=n_list))

source_gurobi = ColumnDataSource(
        data=dict(
            x=range(1, N+1),
            y=solve_time_gurobi,
            obj=solve_time_gurobi,
            desc=n_list))


plot_solve_time = figure(title="Solution Time Comparison", 
                        plot_width=600, plot_height=400, 
                        tools=[TOOLS], y_range=(0,160))

plot_solve_time.circle('x', 'y', size=5, color='red', source=source_cplex, legend='CPLEX')
plot_solve_time.line('x', 'y', 
                 color="#ff4d4d", alpha=0.2, line_width=2, source=source_cplex, legend='CPLEX')

plot_solve_time.circle('x', 'y', size=5, color='green', source=source_gurobi, legend='Gurobi')
plot_solve_time.line('x', 'y', 
                 color='#00b300', alpha=0.2, line_width=2, source=source_gurobi, legend='Gurobi')


plot_solve_time.xaxis.axis_label = '[p = n]'
plot_solve_time.yaxis.axis_label = 'Seconds'

one_quarter = BoxAnnotation(plot=plot_solve_time, top=.35, 
                            fill_alpha=0.1, fill_color='green')
half = BoxAnnotation(plot=plot_solve_time, bottom=.35, top=.7, 
                            fill_alpha=0.1, fill_color='blue')
three_quarter = BoxAnnotation(plot=plot_solve_time, bottom=.7, top=1.05,
                            fill_alpha=0.1, fill_color='gray')

plot_solve_time.renderers.extend([one_quarter, half, three_quarter])
show(plot_solve_time)

-----